# Atmospheric Turbulence

Simulation of atmospheric turbulence's effects on point image source

From http://www.roe.ac.uk/roe/workshop/2011/presentations/tue-pm3-richard-wilson.pdf

- Create 2D array of random complex numbers, normal distribution, mean=0, variance=1
- Phase power spectrum: $\phi(k) = 0.0229r_{0}^{\frac{5}{3}}k^{\frac{-11}{3}}$
- Multiply random numbers by $\sqrt{\phi(k)}$
- FFT to get array of phase fluctuations

Following Matlab example simulation here:  http://www.activeopticalsystems.com/docs/AN021_Kolmogorov%20Spectrum%20Turbulence%20Analysis.pdf

In [ ]:
# Includes
import matplotlib.pyplot as plt
import numpy as np
from scipy.fftpack import fft2, ifft2, fftshift
from scipy.signal import fftconvolve
from scipy.signal import argrelextrema

%matplotlib inline

## Parameters
# Image specs
nxy = 512
center = int(nxy/2)

## Creating Binary Star Input Image
# For KP 2.1m telescope, input image is in units of 25.44 milliarcseconds
platescale = 0.02544 # Plate scale in arcsec/pixel
rho = 2 # Set separation in arcseconds
phi = 30 # Set angle in degrees
# Calculate coordinates of stars
x = int( rho/(2*platescale) * np.cos(np.deg2rad(phi)) )
y = int( rho/(2*platescale) * np.sin(np.deg2rad(phi)) )
x1 = center + x
y1 = center + y
x2 = center - x
y2 = center - y    
# Empty input image
input_img = np.zeros((nxy,nxy)) 
# Place stars on image
input_img[y1,x1] = 1 
input_img[y2,x2] = 1
# Scale image power to 1
input_img_power = np.sum(np.power(input_img,2))
input_img = np.divide(input_img,np.sqrt(input_img_power))

# Aperture/Telescope Specifications:
diameter_m = 2.133 # Mirror diameter in meters
focal_length = 129.69 # Effective focal length in meters
wavelength = 0.8E-6 # Wavelength of light
pixel = 8E-6 # Dimension of a pixel

## Phase screen creation:
# Spectral generation
# Total sample array size
d_aperture = 2.133
# Fried parameter [m]
r0 = 0.15
# Spatial sample resolution
dxy = d_aperture/nxy
# Spatial frequency resolution
df = 1/(d_aperture) 
# Image sample indices array
x = np.multiply( np.subtract(np.arange(nxy),int(nxy/2)), dxy )
# Spatial Frequency indices array
xf = np.multiply( np.subtract(np.arange(nxy),int(nxy/2)), df )
# Meshgrid of spatial frequency domain
[xx,yy]=np.meshgrid(xf,xf)
# Radius from center meshgrid
rr = (np.sqrt(np.power(xx,2)+np.power(yy,2)))
# Calculate Kolmogorov spectral density
alpha = 1
PSD = np.power(rr,-11/3)
PSD = np.multiply(alpha*0.023/(r0**(5/3)),PSD)
# Set DC component to 0 (previous calc attempts to set to 1/0)
PSD[int(nxy/2),int(nxy/2)] = 0 
# Generate Random noise image
random = np.random.normal(0,1,(nxy,nxy)) + 1j*np.random.normal(0,1,(nxy,nxy))  
# Filter random noise image
phase = np.real(ifft2(fftshift(np.sqrt(PSD)*random)*nxy*nxy))
# Create complex phase screen
atmosphere_screen = np.exp(np.multiply(1j,phase))

## Telescope aperture creation:
# Total spatial sample range
X_aperture_s = 1/pixel 
# dx value for sampled aperture image
dx_aperture_s = X_aperture_s/nxy 
# Coordinates of sampled image
x_aperture_s = np.arange(0,X_aperture_s,dx_aperture_s) - X_aperture_s/2
# Meshgrid of sampled coordinates
xx_s,yy_s = np.meshgrid(x_aperture_s,x_aperture_s)
# Scaled aperture diameter to effectively resample aperture image
diameter_s = diameter_m/(focal_length*wavelength)
# Draw new circle at correct dimensions
# Calculate grid of distances from circle center
circle_s = (xx_s) ** 2 + (yy_s) ** 2 
# Draw boolean circle
circle_s = circle_s < (diameter_s/2)**2 
# Convert boolean circle to int
circle_s= circle_s.astype(np.int64)
# Save aperture image in units of meters
aperture_screen_s = circle_s
# Scale aperture image power to 1
aperture_screen_power = np.sum(np.power(aperture_screen_s,2))
aperture_screen_s = np.divide(aperture_screen_s,np.sqrt(aperture_screen_power))
# Calculate effective size of sampled aperture image in meters
X_aperture_s_meff = focal_length*wavelength/pixel

## Calculate Aperture Diffraction pattern
# Calculate normalized complex diffraction pattern
aperture_psf = fftshift(fft2(aperture_screen_s))
# Calculate PSF of complex diffraction pattern
aperture_psf = np.power(np.abs(aperture_psf),2)
# Scale PSF power to 1
aperture_psf_power = np.sum(np.power(aperture_psf,2))
aperture_psf = np.divide(aperture_psf,np.sqrt(aperture_psf_power))

## Calculate Atmospheric Diffraction pattern
# Calculate normalized complex diffraction pattern
atmosphere_psf = fftshift(fft2(atmosphere_screen))
# Calculate PSF of complex diffraction pattern
atmosphere_psf = np.power(np.abs(atmosphere_psf),2)
# Scale PSF power to 1
atmosphere_psf_power = np.sum(np.power(atmosphere_psf,2))
atmosphere_psf = np.divide(atmosphere_psf,np.sqrt(atmosphere_psf_power))

## Calculate system's total response 
# Calculate total PSF of system
psf = fftconvolve(atmosphere_psf,aperture_psf)
# Save center 512x512 image
psf = psf[center:center+nxy,center:center+nxy] 

# Convolve atmosphere PSF with input image using FFT
sensor_img = fftconvolve(input_img,psf)
# Save the center 512x512 image
sensor_img = sensor_img[center:center+nxy,center:center+nxy] 

##Plots
colormap = "jet"

plt.figure(figsize = (14,6), dpi = 100)
plt.subplot(1,2,1)
plt.imshow(np.angle(atmosphere_screen), cmap=colormap)
plt.colorbar()
plt.title("Atmosphere Phase Shift")
plt.subplot(1,2,2)
plt.imshow(atmosphere_psf, cmap=colormap)
plt.xlabel("[pixels]")
plt.ylabel("[pixels]")
plt.title("Atmosphere PSF")

plt.figure(figsize = (14,6), dpi = 100)
plt.subplot(1,2,1)
plt.imshow(aperture_screen_s, cmap=colormap, 
           extent = (-X_aperture_s_meff/2,X_aperture_s_meff/2,
                     -X_aperture_s_meff/2,X_aperture_s_meff/2))
plt.title("Aperture Magnitude")
plt.xlabel("[m]")
plt.ylabel("[m]")
plt.subplot(1,2,2)
plt.imshow(aperture_psf, cmap=colormap)
plt.xlabel("[pixels]")
plt.ylabel("[pixels]")
plt.title("Aperture PSF")

plt.figure(figsize = (8,8), dpi = 100)
plt.imshow(psf, cmap=colormap)
plt.title("Total System PSF")

plt.figure(figsize = (14,6), dpi = 100)
plt.subplot(1,2,1)
plt.imshow(input_img, cmap=colormap, extent = (-6.513,6.513,-6.513,6.513))
plt.xlabel("[arcsec]")
plt.ylabel("[arcsec]")
plt.title("Input Image")
plt.subplot(1,2,2)
plt.imshow(sensor_img, cmap=colormap)
plt.xlabel("[pixels]")
plt.ylabel("[pixels]")
plt.title("Sensor Image")

In [ ]:
# Calculating RMS Phase variation to confirm proper normalization

## Draw a circle to use as a mask
# Calculate scale of phase mask in m/pixel
phase_scale = d_aperture/nxy
# Calculate size of fried parameter in pixels
fried_radius_m = r0/2
fried_radius_p = fried_radius_m/phase_scale
# Draw circular mask
fried_mask = np.zeros((nxy,nxy))
for i in np.arange(nxy):
    for j in np.arange(nxy):
        if np.sqrt((i-center)**2+(j-center)**2) <= fried_radius_p:
            fried_mask[i,j] = 1

## Mask phase mask and calculate RMS 
fried_cell = np.multiply(fried_mask,phase)
plt.imshow(fried_cell)
plt.colorbar()

phase_var_s = np.power(fried_cell,2)
phase_var_rms = np.power(np.mean(phase_var_s),0.5)
print(phase_var_rms)

plt.imshow(phase)
            
